In [1]:
# ============================================================
# IMPORT LIBRARY
# ============================================================

# --- Library Dasar & Manajemen File ---
import os                       # Manajemen path file/folder dataset
import glob                     # Pencarian file dengan pola tertentu (misalnya *.jpg)
import random                   # Membuat nilai acak (misalnya shuffle dataset)
import pandas as pd             # Manipulasi data tabular (CSV, DataFrame)
import numpy as np              # Operasi matriks & array (fondasi AI/ML)
import math                     # Fungsi matematika kompleks (misalnya log, gamma)
import time                     # Mengukur durasi training (timer)

# --- Library Visualisasi (Grafik & Plot) ---
import matplotlib.pyplot as plt # Membuat grafik akurasi, loss, dll.
import seaborn as sns           # Visualisasi heatmap (misalnya confusion matrix)

# --- Library Pemrosesan Gambar (Preprocessing) ---
import cv2                      # OpenCV: resize, noise removal, membaca gambar
from PIL import Image           # Alternatif manipulasi gambar sederhana
from sklearn.model_selection import train_test_split  # Membagi dataset (train/test split)
from sklearn.model_selection import KFold             # K-Fold Cross Validation

# --- Library Deep Learning (Keras/TensorFlow) ---
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model # Membuat model sequential atau custom
from tensorflow.keras.layers import (
    Conv2D,             # Layer konvolusi untuk ekstraksi fitur
    MaxPooling2D,       # Layer pooling untuk reduksi dimensi
    Flatten,            # Meratakan array sebelum masuk ke fully connected layer
    Dense,              # Fully connected layer (klasifikasi akhir)
    Dropout,            # Regularisasi untuk mencegah overfitting
    Input,              # Definisi input layer
    BatchNormalization, # Normalisasi batch agar training stabil
    Activation          # Fungsi aktivasi (ReLU, Softmax, dll.)
)
from tensorflow.keras.optimizers import SGD            # Optimizer dasar (bisa dituning)
from tensorflow.keras.regularizers import l2           # Regularisasi L2 (mencegah overfitting)

# --- Library Evaluasi Performa ---
from sklearn.metrics import (
    confusion_matrix,       # Matriks kesalahan prediksi per kelas
    classification_report,  # Precision, Recall, F1-Score
    accuracy_score,         # Akurasi sederhana
    roc_curve,              # Kurva ROC (Receiver Operating Characteristic)
    auc                     # Area Under Curve (AUC)
)

# --- Utility Tambahan ---
from tensorflow.keras.preprocessing.image import ImageDataGenerator # Augmentasi data gambar
from tensorflow.keras.utils import to_categorical                   # One-hot encoding label

# ============================================================
# CEK KETERSEDIAAN GPU
# ============================================================
print(f"Versi TensorFlow: {tf.__version__}")  # Menampilkan versi TensorFlow
print("GPU tersedia: ", "Ya" if tf.config.list_physical_devices('GPU') else "Tidak")  # Mengecek apakah GPU aktif

2025-11-24 01:59:00.705848: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763949540.922523      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763949540.991688      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

Versi TensorFlow: 2.18.0
GPU tersedia:  Ya


In [2]:
# 1. Matikan Warning Python
warnings.filterwarnings("ignore")

# 2. Matikan Log TensorFlow yang berisik (INFO & WARNING)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 
# Level 0 = Semua log muncul
# Level 1 = Filter INFO
# Level 2 = Filter INFO & WARNING (Kita pilih ini)
# Level 3 = Filter ERROR juga

print("✅ Log Warning berhasil disembunyikan.")

NameError: name 'warnings' is not defined

In [ ]:
# ============================================================
# BAGIAN 1: PERSIAPAN DATASET (MODIFIKASI 4 KELAS)
# ============================================================
# Dokumentasi:
# Kode ini menggabungkan seluruh data (Training + Testing bawaan dataset)
# menjadi satu, lalu split manual 70:30 sesuai metodologi Paper.

print("\n--- MENYIAPKAN DATASET 4 KELAS ---")   # Info proses dimulai

# 1. Tentukan Lokasi Dataset 
DATASET_ROOT = '/kaggle/input/tumorv2'   # Path root dataset (ubah sesuai lokasi dataset)

filepaths = []   # List untuk menyimpan path gambar
labels = []      # List untuk menyimpan label kelas

# 2. Crawling Data (Mencari semua file gambar secara rekursif)
# Kita mencari ke dalam folder 'Training' dan 'Testing' sekaligus
for category in ['Training', 'Testing']:                  # Loop ke folder Training & Testing
    path = os.path.join(DATASET_ROOT, category)           # Gabungkan path root + kategori
    
    if os.path.exists(path):                              # Jika folder ada
        classes = os.listdir(path)                        # Ambil daftar sub-folder (nama kelas)
        
        for class_name in classes:                        # Loop tiap kelas
            class_dir = os.path.join(path, class_name)    # Path ke folder kelas
            if os.path.isdir(class_dir):                  # Pastikan benar folder
                files = glob.glob(os.path.join(class_dir, '*'))  # Ambil semua file gambar
                for f in files:                           # Loop tiap file gambar
                    filepaths.append(f)                   # Simpan path gambar
                    labels.append(class_name)             # Simpan label kelas
    else:
        # Jika struktur folder tidak punya 'Training/Testing', coba baca langsung
        direct_path = DATASET_ROOT
        if os.path.exists(direct_path):                   # Jika folder root ada
            classes = [d for d in os.listdir(direct_path) 
                       if os.path.isdir(os.path.join(direct_path, d))]  # Ambil sub-folder kelas
            for class_name in classes:
                if class_name in ['Training', 'Testing']: continue      # Skip jika folder Training/Testing
                class_dir = os.path.join(direct_path, class_name)       # Path ke folder kelas
                files = glob.glob(os.path.join(class_dir, '*'))         # Ambil semua file gambar
                for f in files:
                    filepaths.append(f)                   # Simpan path gambar
                    labels.append(class_name)             # Simpan label kelas

# 3. Buat DataFrame & Cek Data
df = pd.DataFrame({'filename': filepaths, 'class': labels})   # Buat dataframe dengan kolom filename & class
df = df.drop_duplicates(subset=['filename'])                  # Hapus duplikat jika ada

# 4. Acak Data (SHUFFLE) - Sangat Penting!
df = df.sample(frac=1, random_state=42).reset_index(drop=True)  # Shuffle data agar distribusi acak

# 5. Info Dataset
NUM_CLASSES = len(df['class'].unique())                        # Hitung jumlah kelas unik
print(f"Total Data Ditemukan : {len(df)} gambar")              # Total gambar
print(f"Jumlah Kelas         : {NUM_CLASSES}")                 # Jumlah kelas
print(f"Nama Kelas           : {df['class'].unique()}")        # Nama kelas

# 6. SPLIT MANUAL 70:30 (Sesuai Paper)
if len(df) > 0:
    split_idx = int(len(df) * 0.70)                            # Hitung index split 70%
    df_train_full = df.iloc[:split_idx]                        # 70% untuk training (optimasi K-Fold)
    df_test_final = df.iloc[split_idx:]                        # 30% untuk final testing
    
    print(f"\n--- STATUS PEMBAGIAN DATA (PAPER SPLIT) ---")
    print(f"Data Training (70%) : {len(df_train_full)} -> Masuk ke 5-Fold CV")
    print(f"Data Testing (30%)  : {len(df_test_final)} -> Disimpan untuk Evaluasi Akhir")
else:
    print("❌ ERROR: Tidak ada gambar ditemukan. Cek path 'DATASET_ROOT' Anda!")  # Jika dataset kosong

In [ ]:
# ============================================================
# VISUALISASI SAMPEL DATASET (4 KELAS)
# ============================================================

# 1. Pastikan data sudah siap
if 'df_train_full' in globals():                     # Mengecek apakah variabel df_train_full sudah ada
    print("--- MENAMPILKAN SAMPEL GAMBAR ---")       # Info bahwa proses visualisasi dimulai

    # 2. Buat Generator Sementara (Hanya untuk ambil sampel)
    viz_datagen = ImageDataGenerator(rescale=1./255) # Normalisasi pixel ke [0,1] agar lebih stabil

    viz_generator = viz_datagen.flow_from_dataframe( # Membuat generator dari dataframe
        df_train_full,                               # Dataframe berisi path file & label
        x_col='filename',                            # Kolom berisi path gambar
        y_col='class',                               # Kolom berisi label kelas
        target_size=(128, 128),                      # Resize gambar ke ukuran 128x128
        batch_size=9,                                # Ambil 9 gambar sekaligus
        class_mode='categorical',                    # Label dalam bentuk one-hot encoding
        shuffle=True                                 # Acak urutan gambar agar variatif
    )

    # 3. Ambil 1 Batch Data
    try:
        images, labels = next(viz_generator)         # Ambil batch pertama (9 gambar + label)

        # Buat Mapping Index ke Nama Kelas
        # class_indices: {'glioma':0, 'meningioma':1, ...}
        # Dibalik jadi {0:'glioma', 1:'meningioma', ...}
        idx_to_label = {v: k for k, v in viz_generator.class_indices.items()}

        # 4. Plotting
        plt.figure(figsize=(12, 12))                 # Ukuran canvas 12x12 inch

        for i in range(min(9, len(images))):         # Loop untuk 9 gambar
            ax = plt.subplot(3, 3, i + 1)            # Buat grid 3x3

            plt.imshow(images[i])                    # Tampilkan gambar ke subplot

            # Ambil Label dari one-hot encoding (contoh: [0,0,1,0] → index 2)
            class_idx = np.argmax(labels[i])         # Cari index dengan nilai 1
            class_name = idx_to_label[class_idx]     # Konversi index ke nama kelas

            plt.title(class_name)                    # Judul subplot = nama kelas
            plt.axis("off")                          # Hilangkan axis agar lebih bersih

        plt.show()                                   # Tampilkan semua gambar

    except Exception as e:
        print(f"Gagal menampilkan gambar: {e}")      # Jika error, tampilkan pesan

else:
    print("⚠️ ERROR: Variabel 'df_train_full' belum ada.") # Jika dataset belum disiapkan
    print("Silakan jalankan kode 'BAGIAN 2: PERSIAPAN DATASET' terlebih dahulu.")

In [ ]:
# ============================================================
# BAGIAN 2: HELPER MATH (NLCMFO UTILS)
# ============================================================
# Dokumentasi:
# Implementasi rumus matematika dari Paper.
# 1. levy_flight: Membuat pergerakan 'lompatan jauh' acak untuk eksplorasi.
# 2. get_chaotic_value: Peta kekacauan (Sine/Chebyshev) untuk parameter acak.
# 3. create_flexible_cnn: Arsitektur CNN ringan (6 layer) sesuai Paper.

def levy_flight(beta=1.5):
    # Rumus Levy Flight untuk eksplorasi acak
    num = math.gamma(1 + beta) * math.sin(math.pi * beta / 2)   # Bagian numerator
    den = math.gamma((1 + beta) / 2) * beta * (2 ** ((beta - 1) / 2))  # Bagian denominator
    sigma_x = (num / den) ** (1 / beta)                         # Skala distribusi
    u = np.random.normal(0, sigma_x)                            # Sampel normal dengan sigma_x
    v = np.random.normal(0, 1)                                  # Sampel normal standar
    step = 0.05 * u / (abs(v) ** (1 / beta))                    # Langkah Levy Flight
    return step                                                 # Return nilai step

def get_chaotic_value(map_type, x_old):
    # Fungsi untuk menghasilkan nilai chaotic (sine/chebyshev map)
    x_new = 0
    if map_type == 'sine':                                      # Peta sine
        x_new = (4.0 / 4.0) * math.sin(math.pi * x_old)
    elif map_type == 'chebyshev':                               # Peta chebyshev
        x_new = math.cos(5 * math.acos(x_old))
    if x_new == 0: x_new = 0.0001                               # Hindari nilai nol
    return abs(x_new)                                           # Return nilai absolut

def create_flexible_cnn(width=128, height=128, depth=3, classes=2, l2_reg=0.0001):
    """
    Arsitektur Lightweight CNN 6 Layer (Sesuai Paper)
    """
    model = Sequential()
    inputShape = (height, width, depth)                         # Bentuk input gambar
    chanDim = -1                                                # Channel terakhir (format TensorFlow)

    model.add(Input(shape=inputShape))                          # Input layer

    # Blok 1
    model.add(Conv2D(32, (3, 3), padding="same", kernel_regularizer=l2(l2_reg))) # Conv layer 32 filter
    model.add(BatchNormalization(axis=chanDim)); model.add(Activation("relu"))   # Normalisasi + ReLU
    model.add(Conv2D(32, (3, 3), padding="same", kernel_regularizer=l2(l2_reg))) # Conv layer kedua
    model.add(BatchNormalization(axis=chanDim)); model.add(Activation("relu"))
    model.add(MaxPooling2D(pool_size=(2, 2))); model.add(Dropout(0.25))          # Pooling + Dropout

    # Blok 2
    model.add(Conv2D(64, (3, 3), padding="same", kernel_regularizer=l2(l2_reg))) # Conv layer 64 filter
    model.add(BatchNormalization(axis=chanDim)); model.add(Activation("relu"))
    model.add(Conv2D(64, (3, 3), padding="same", kernel_regularizer=l2(l2_reg)))
    model.add(BatchNormalization(axis=chanDim)); model.add(Activation("relu"))
    model.add(MaxPooling2D(pool_size=(2, 2))); model.add(Dropout(0.25))

    # Blok 3
    model.add(Conv2D(128, (3, 3), padding="same", kernel_regularizer=l2(l2_reg))) # Conv layer 128 filter
    model.add(BatchNormalization(axis=chanDim)); model.add(Activation("relu"))
    model.add(Conv2D(128, (3, 3), padding="same", kernel_regularizer=l2(l2_reg)))
    model.add(BatchNormalization(axis=chanDim)); model.add(Activation("relu"))
    model.add(MaxPooling2D(pool_size=(2, 2))); model.add(Dropout(0.25))

    # Classifier (Fully Connected)
    model.add(Flatten())                                          # Meratakan output conv
    model.add(Dense(512, kernel_regularizer=l2(l2_reg)))          # Dense layer 512 neuron
    model.add(BatchNormalization()); model.add(Activation("relu")); model.add(Dropout(0.5))
    model.add(Dense(classes)); model.add(Activation("softmax"))   # Output layer sesuai jumlah kelas

    return model                                                  # Return model CNN

In [ ]:
# ============================================================
# BAGIAN 3: OBJECTIVE FUNCTION (DENGAN EARLY STOPPING)
# ============================================================
def objective_function_kfold(hyperparameters):
    global df_train_full, NUM_CLASSES   # Gunakan dataset training & jumlah kelas dari variabel global
    
    # Decode Parameter (hyperparameter yang dioptimasi)
    lr_val = hyperparameters[0]         # Learning rate
    mom_val = hyperparameters[1]        # Momentum untuk SGD
    epochs_val = int(hyperparameters[2])# Jumlah maksimum epoch (dibatasi 50)
    l2_val = hyperparameters[3]         # Nilai regularisasi L2
    
    kfold = KFold(n_splits=5, shuffle=True, random_state=42)  # K-Fold Cross Validation (5 fold)
    fold_accuracies = []                # List untuk menyimpan akurasi tiap fold
    
    print(f"\n>> [UJI PARAMETER] LR={lr_val:.4f}, Mom={mom_val:.4f}, MaxEp={epochs_val}, L2={l2_val:.4f}")
    
    # --- DEFINISI EARLY STOPPING ---
    # Strategi: hentikan training jika akurasi validasi stagnan 5 epoch
    early_stop = EarlyStopping(
        monitor='val_accuracy',         # Pantau akurasi validasi
        patience=5,                     # Jika stagnan 5 epoch → stop
        restore_best_weights=True,      # Ambil bobot terbaik, bukan bobot terakhir
        mode='max',                     # Target: akurasi maksimum
        verbose=0                       # Silent mode (tidak print log)
    )
    
    # Loop K-Fold
    for fold_idx, (train_idx, val_idx) in enumerate(kfold.split(df_train_full)):
        
        tf.keras.backend.clear_session()    # Reset RAM/VRAM sebelum fold dimulai
        
        train_data = df_train_full.iloc[train_idx]   # Data training untuk fold ini
        val_data   = df_train_full.iloc[val_idx]     # Data validasi untuk fold ini
        
        # Generator untuk training (augmentasi gambar)
        train_gen = ImageDataGenerator(
            rescale=1./255, rotation_range=30, width_shift_range=0.1,
            height_shift_range=0.1, horizontal_flip=True, fill_mode='nearest'
        ).flow_from_dataframe(
            train_data, x_col='filename', y_col='class',
            target_size=(128, 128), batch_size=32,
            class_mode='categorical', shuffle=True, verbose=0
        )
        
        # Generator untuk validasi (hanya rescale, tanpa augmentasi)
        val_gen = ImageDataGenerator(rescale=1./255).flow_from_dataframe(
            val_data, x_col='filename', y_col='class',
            target_size=(128, 128), batch_size=32,
            class_mode='categorical', shuffle=False, verbose=0
        )
        
        # Build Model CNN sesuai arsitektur flexible
        model = create_flexible_cnn(classes=NUM_CLASSES, l2_reg=l2_val)
        opt   = SGD(learning_rate=lr_val, momentum=mom_val)     # Optimizer SGD dengan LR & momentum
        model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])
        
        try:
            # Train model dengan Early Stopping
            history = model.fit(
                train_gen, steps_per_epoch=len(train_gen),
                validation_data=val_gen, validation_steps=len(val_gen),
                epochs=epochs_val,
                callbacks=[early_stop],    # Pasang Early Stopping di sini
                verbose=0                  # Silent mode
            )
            best_acc = max(history.history['val_accuracy'])      # Ambil akurasi validasi terbaik
            actual_epoch = len(history.history['loss'])          # Epoch terakhir yang dijalankan
            # print(f"   (Fold {fold_idx+1} stop di epoch {actual_epoch}/{epochs_val})")
        
        except Exception as e:
            print(f"Error: {e}"); best_acc = 0.0                 # Jika error, akurasi = 0
        
        fold_accuracies.append(best_acc)                        # Simpan akurasi fold ini
        
        # Bersihkan memori tiap fold
        del model, history, train_gen, val_gen
        tf.keras.backend.clear_session()
        gc.collect()
    
    # Hitung rata-rata akurasi semua fold
    avg_acc = np.mean(fold_accuracies)
    error_rate = (1.0 - avg_acc) * 100                          # Error rate = 1 - akurasi
    
    print(f"   >> Avg Acc: {avg_acc*100:.2f}% | Error: {error_rate:.2f}%")
    gc.collect()
    
    return error_rate                                           # Return error rate untuk optimasi

In [ ]:
# ============================================================
# CEK DATA VISUALISASI (JALANKAN INI UNTUK MELIHAT GAMBAR)
# ============================================================

# 1. Pastikan DataFrame sudah ada
if 'df_train_full' in globals():                           # Mengecek apakah variabel df_train_full sudah tersedia
    print("--- MENAMPILKAN SAMPEL DATA TRAINING (RGB + AUGMENTASI) ---")

    # 2. Buat Generator Persis seperti saat Training
    # Menggunakan parameter augmentasi yang sama (rotasi, shift, flip)
    check_datagen = ImageDataGenerator(
        rescale=1./255,                                    # Normalisasi pixel ke [0,1]
        rotation_range=30,                                 # Rotasi acak hingga 30 derajat
        width_shift_range=0.1,                             # Pergeseran horizontal 10%
        height_shift_range=0.1,                            # Pergeseran vertikal 10%
        horizontal_flip=True,                              # Flip horizontal acak
        fill_mode='nearest'                                # Isi piksel kosong dengan nilai terdekat
    )
    
    check_generator = check_datagen.flow_from_dataframe(
        df_train_full,                                     # DataFrame berisi path & label
        x_col='filename',                                  # Kolom path gambar
        y_col='class',                                     # Kolom label kelas
        target_size=(128, 128),                            # Resize ke 128x128
        batch_size=5,                                      # Ambil 5 gambar saja
        class_mode='categorical',                          # Label dalam bentuk one-hot
        shuffle=True                                       # Acak urutan gambar
    )

    # 3. Ambil 1 Batch
    try:
        images, labels = next(check_generator)             # Ambil batch pertama (5 gambar + label)
        
        # Cek Dimensi (Harusnya (5, 128, 128, 3))
        print(f"\n[INFO TEKNIS] Bentuk Data: {images.shape}")
        if images.shape[-1] == 3:                          # Jika channel terakhir = 3 → RGB
            print("✅ STATUS: Gambar adalah RGB (3 Channel).")
        else:
            print("⚠️ STATUS: Gambar bukan RGB.")

        # 4. Tampilkan Gambar
        plt.figure(figsize=(15, 5))                        # Canvas ukuran 15x5 inch
        
        # Mapping index ke nama kelas (contoh: 0 → glioma)
        idx_to_label = {v: k for k, v in check_generator.class_indices.items()}
        
        for i in range(min(5, len(images))):               # Loop untuk 5 gambar
            ax = plt.subplot(1, 5, i + 1)                  # Buat grid 1 baris, 5 kolom
            
            plt.imshow(images[i])                          # Tampilkan gambar
            
            class_idx = np.argmax(labels[i])               # Ambil index kelas dari one-hot
            class_name = idx_to_label[class_idx]           # Konversi index ke nama kelas
            
            plt.title(f"{class_name}\n{images[i].shape}")  # Judul = nama kelas + dimensi gambar
            plt.axis("off")                                # Hilangkan axis agar lebih bersih
            
        plt.tight_layout()                                 # Atur layout agar rapi
        plt.show()                                         # Tampilkan semua gambar
        
        print("\nKeterangan:")
        print("- Jika gambar terlihat miring/terpotong, itu efek AUGMENTASI (Rotation/Shift).")
        print("- Jika gambar berwarna, berarti mode RGB aktif.")
        
    except Exception as e:
        print(f"Gagal menampilkan gambar: {e}")            # Jika error, tampilkan pesan

else:
    print("⚠️ ERROR: Variabel 'df_train_full' belum ada.") # Jika dataset belum disiapkan
    print("Silakan jalankan cell 'BAGIAN 2: PERSIAPAN DATASET' terlebih dahulu.")

In [ ]:
# ============================================================
# BAGIAN 4: ALGORITMA NLCMFO (DENGAN MEMORY MANAGEMENT)
# ============================================================
def NLCMFO(objf, lb, ub, dim, N, Max_iteration):
    Moth_pos = np.zeros((N, dim))                          # Matriks posisi awal populasi (N ngengat, dim dimensi)
    # Inisialisasi posisi awal acak
    for i in range(dim): 
        Moth_pos[:, i] = np.random.uniform(0, 1, N) * (ub[i] - lb[i]) + lb[i]  # Posisi acak dalam batas lb-ub
        
    Moth_fitness = np.full(N, float("inf"))                # Fitness awal (inf → belum dihitung)
    best_flames = np.copy(Moth_pos)                        # Simpan posisi terbaik (flame)
    best_flame_fitness = np.zeros(N)                       # Fitness flame terbaik
    chaos_val = 0.7                                        # Nilai awal chaos
    last_best_score = float("inf")                         # Skor terbaik terakhir
    stagnation_counter = 0                                 # Counter stagnasi
    current_map = 'sine'                                   # Map chaos default
    
    Iteration = 1
    while Iteration <= Max_iteration:                      # Loop iterasi utama
        Flame_no = round(N - Iteration * ((N - 1) / Max_iteration))  # Jumlah flame aktif
        print(f"\n=== NLCMFO ITERASI {Iteration}/{Max_iteration} ===")
        
        # --- EVALUASI POPULASI (Perbaikan Memory Management) ---
        for i in range(N):
            print(f"  > Mengevaluasi Moth {i+1} dari {N}...", end="")
            
            # Clamping batasan (agar posisi tetap dalam range lb-ub)
            for j in range(dim): 
                Moth_pos[i, j] = np.clip(Moth_pos[i, j], lb[j], ub[j])
            
            # Jalankan Objective Function untuk menghitung fitness
            Moth_fitness[i] = objf(Moth_pos[i, :])
            
            # Bersihkan RAM setelah evaluasi satu ngengat
            tf.keras.backend.clear_session()
            gc.collect()
            # time.sleep(1) # Opsional: jeda 1 detik agar OS reclaim memory
        
        # --- Sorting & Update Flames ---
        if Iteration == 1:                                 # Iterasi pertama → langsung sort
            I = np.argsort(Moth_fitness)                   # Urutkan fitness
            sorted_population = Moth_pos[I, :]             # Populasi terurut
            fitness_sorted = np.sort(Moth_fitness)         # Fitness terurut
            best_flames = sorted_population                # Simpan flame terbaik
            best_flame_fitness = fitness_sorted
        else:                                              # Iterasi berikutnya → gabungkan populasi lama & flame
            double_pop = np.concatenate((Moth_pos, best_flames), axis=0)
            double_fit = np.concatenate((Moth_fitness, best_flame_fitness), axis=0)
            I2 = np.argsort(double_fit)                    # Urutkan gabungan
            best_flames = double_pop[I2, :][:N]            # Ambil N terbaik
            best_flame_fitness = double_fit[I2][:N]
            sorted_population = best_flames
        
        Best_flame_score = best_flame_fitness[0]           # Fitness terbaik saat ini
        Best_flame_pos = best_flames[0, :]                 # Posisi terbaik saat ini
        
        # --- Chaotic Switching ---
        if abs(Best_flame_score - last_best_score) < 1e-6: # Jika skor stagnan
            stagnation_counter += 1
        else:                                              # Jika ada perbaikan
            stagnation_counter = 0
            last_best_score = Best_flame_score
            current_map = 'sine'                           # Reset ke sine map
        if stagnation_counter >= 3:                        # Jika stagnasi 3 kali berturut-turut
            current_map = 'chebyshev'                      # Ganti ke chebyshev map
            print("  [INFO] Stagnasi! Ganti Map.")
        
        chaos_val = get_chaotic_value(current_map, chaos_val)  # Update nilai chaos
        a = -1 + Iteration * ((-1) / Max_iteration)            # Parameter a (kontrol spiral)
        w = 2 * math.exp(-((6 * Iteration / Max_iteration) ** 2))  # Parameter w (kontrol bobot)
        
        # --- Update Posisi Moth ---
        for i in range(N):
            for j in range(dim):
                if i <= Flame_no:                            # Jika masih dalam jumlah flame aktif
                    target = sorted_population[i, j]         # Target = flame ke-i
                    dist = abs(sorted_population[i, j] - Moth_pos[i, j])
                else:                                        # Jika di luar flame aktif
                    target = sorted_population[Flame_no, j]  # Target = flame terakhir
                    dist = abs(sorted_population[Flame_no, j] - Moth_pos[i, j])
                t = abs((a - 1) * chaos_val + 1)             # Parameter spiral
                LF = levy_flight(beta=1.5)                   # Levy Flight untuk eksplorasi
                spiral = dist * math.exp(1 * t) * math.cos(t * 2 * math.pi)  # Rumus spiral
                Moth_pos[i, j] = (w * LF * spiral) + (LF * target)           # Update posisi
                
        print(f"  >> Best Error Rate Iterasi ini: {Best_flame_score:.4f}%")
        Iteration += 1
        
        # Bersihkan memori setelah satu iterasi penuh
        gc.collect()
    
    return Best_flame_pos, Best_flame_score                 # Return posisi & skor terbaik

In [ ]:
# ============================================================
# BAGIAN 7: EKSEKUSI UTAMA (SETTING 4000 DATA)
# ============================================================

if len(df) > 0:                                           # Pastikan dataset tidak kosong
    # 1. Konfigurasi Batas Parameter
    lb = [0.001, 0.1, 10, 0.0001]                         # Lower bound: LR, Momentum, Epochs, L2
    ub = [0.1, 0.99, 50, 0.01]                            # Upper bound: LR, Momentum, Epochs, L2

    # 2. Setting Iterasi 
    N_population = 10                                     # Jumlah populasi (ngengat)
    Max_iterations = 1                                    # Jumlah iterasi optimasi
    
    print("\n=== MEMULAI PROSES OPTIMASI (NLCMFO + EARLY STOPPING) ===")
    print(f"Dataset Size: {len(df)} images")
    print(f"Population: {N_population} | Max Epoch per Training: 50 (dengan Patience=5)")
    
    # Bersihkan RAM Awal
    tf.keras.backend.clear_session()                      # Reset session TensorFlow
    gc.collect()                                          # Garbage collector
    
    start_time = time.time()                              # Catat waktu mulai
    
    # Jalankan NLCMFO (optimasi hyperparameter)
    best_pos, best_score = NLCMFO(objective_function_kfold, lb, ub, 4, N_population, Max_iterations)
    
    end_time = time.time()                                # Catat waktu selesai

    print(f"\n=== HASIL OPTIMASI SELESAI ===")
    print(f"Durasi: {(end_time - start_time)/60:.2f} Menit")   # Lama proses optimasi
    print(f"Parameter Terbaik:")                               # Parameter hasil optimasi
    print(f" - LR      : {best_pos[0]:.5f}")
    print(f" - Momentum: {best_pos[1]:.5f}")
    print(f" - Epochs  : {int(best_pos[2])}")
    print(f" - L2 Reg  : {best_pos[3]:.5f}")

    # ============================================================
    # FINAL TEST: TRAIN ULANG DENGAN CHECKPOINT & EARLY STOPPING
    # ============================================================
    print("\n--- TRAINING MODEL FINAL (70% TRAIN DATA) ---")
    
    # Bersihkan RAM sebelum Final Train
    tf.keras.backend.clear_session()
    gc.collect()
    time.sleep(2)                                         # Jeda 2 detik agar RAM stabil

    # Generator untuk training final (augmentasi ringan)
    final_train_gen = ImageDataGenerator(
        rescale=1./255, rotation_range=30, fill_mode='nearest'
    ).flow_from_dataframe(
        df_train_full, x_col='filename', y_col='class',
        target_size=(128, 128), batch_size=32, 
        class_mode='categorical'
    )
    
    # Generator untuk testing final (hanya rescale)
    final_test_gen = ImageDataGenerator(rescale=1./255).flow_from_dataframe(
        df_test_final, x_col='filename', y_col='class',
        target_size=(128, 128), batch_size=32, 
        class_mode='categorical', shuffle=False
    )

    # Bangun model CNN dengan parameter terbaik hasil optimasi
    model_final = create_flexible_cnn(classes=NUM_CLASSES, l2_reg=best_pos[3])
    model_final.compile(
        loss="categorical_crossentropy",
        optimizer=SGD(learning_rate=best_pos[0], momentum=best_pos[1]),
        metrics=["accuracy"]
    )

    # Checkpoint + Early Stopping untuk Final Model
    checkpoint_path = "Best_Model_CNN_NLCMFO.h5"          # Simpan model terbaik
    
    callbacks_list = [
        ModelCheckpoint(checkpoint_path, monitor='val_accuracy', mode='max', save_best_only=True, verbose=1),
        EarlyStopping(monitor='val_accuracy', patience=8, restore_best_weights=True, verbose=1) # Patience lebih longgar
    ]

    # Train Final Model
    history = model_final.fit(
        final_train_gen, 
        epochs=int(best_pos[2]), 
        validation_data=final_test_gen,
        callbacks=callbacks_list,
        verbose=1
    )

    # Load Best Model dari checkpoint
    print("\n--- MEMUAT KEMBALI BOBOT MODEL TERBAIK ---")
    try:
        model_final.load_weights(checkpoint_path)         # Muat bobot terbaik
        print("✅ Berhasil memuat model terbaik.")
    except:
        print("⚠️ Load gagal, menggunakan model terakhir.")

    # Evaluasi Akhir pada data test (30% unseen)
    print("\n--- EVALUASI FINAL PADA DATA TEST (30% UNSEEN) ---")
    loss, acc = model_final.evaluate(final_test_gen)      # Evaluasi model
    print(f"AKURASI AKHIR (BEST MODEL): {acc*100:.2f}%")  # Cetak akurasi akhir

else:
    print("Dataset Kosong.")                              # Jika dataset kosong

In [ ]:
# ============================================================
# EKSEKUSI FINAL LANGSUNG (SKIP OPTIMASI)
# ============================================================

# 1. Parameter manual (ambil dari log terbaik sebelumnya)
MANUAL_LR      = 0.08066   # Learning Rate terbaik dari log
MANUAL_MOM     = 0.11968   # Momentum terbaik dari log
MANUAL_EPOCHS  = 50        # Jumlah epoch terbaik dari log
MANUAL_L2      = 0.00180   # Nilai regularisasi L2 terbaik dari log

print("\n=== TRAINING FINAL LANGSUNG ===")
print(f"LR: {MANUAL_LR}, Mom: {MANUAL_MOM}, Ep: {MANUAL_EPOCHS}, L2: {MANUAL_L2}")
print(f"Train Data : {len(df_train_full)} | Test Data : {len(df_test_final)} | Kelas: {NUM_CLASSES}")

# 2. Bersihkan memori sebelum training
tf.keras.backend.clear_session()
gc.collect()

# 3. Generator untuk training & testing
train_gen = ImageDataGenerator(
    rescale=1./255, rotation_range=30, fill_mode='nearest'
).flow_from_dataframe(
    df_train_full, x_col='filename', y_col='class',
    target_size=(128, 128), batch_size=32,
    class_mode='categorical', shuffle=True
)

test_gen = ImageDataGenerator(rescale=1./255).flow_from_dataframe(
    df_test_final, x_col='filename', y_col='class',
    target_size=(128, 128), batch_size=32,
    class_mode='categorical', shuffle=False
)

# 4. Bangun & compile model sekali saja
model_final = create_flexible_cnn(classes=NUM_CLASSES, l2_reg=MANUAL_L2)
model_final.compile(
    loss="categorical_crossentropy",
    optimizer=SGD(learning_rate=MANUAL_LR, momentum=MANUAL_MOM),
    metrics=["accuracy"]
)

# 5. Setup checkpoint
checkpoint = ModelCheckpoint(
    'Best_Model_Saved_Manual.h5',
    monitor='val_accuracy', mode='max',
    save_best_only=True, verbose=1
)

# 6. Training final
history = model_final.fit(
    train_gen,
    epochs=int(MANUAL_EPOCHS),
    validation_data=test_gen,
    callbacks=[checkpoint],
    verbose=1
)

# 7. Evaluasi akhir
print("\n--- HASIL AKHIR ---")
model_final.load_weights('Best_Model_Saved_Manual.h5')   # Load bobot terbaik
loss, acc = model_final.evaluate(test_gen)
print(f"AKURASI FINAL: {acc*100:.2f}%")

In [ ]:
# ============================================================
# 1. LOAD WEIGHTS & PREDIKSI 
# ============================================================

# Muat bobot terbaik (pastikan file ada)
try:
    model_final.load_weights('Best_Model_Saved_Manual.h5')   # Load bobot model terbaik dari file .h5
    print("✅ Bobot model berhasil dimuat.")
except:
    print("⚠️ File bobot tidak ditemukan, menggunakan bobot terakhir di memori.")

# Evaluasi skor dasar pada data test
print("\n--- EVALUASI DATA TEST ---")
loss, acc = model_final.evaluate(final_test_gen, verbose=0)  # Evaluasi model pada generator test
print(f"AKURASI FINAL: {acc*100:.2f}%")                      # Cetak akurasi akhir

# ============================================================
# 2. GENERATE PREDIKSI DETAIL
# ============================================================

print("Sedang menghitung prediksi detail...")

final_test_gen.reset()                                       # Reset generator agar urutan data konsisten

y_pred_probs = model_final.predict(final_test_gen, verbose=1) # Ambil probabilitas prediksi (softmax output)

y_pred_classes = np.argmax(y_pred_probs, axis=1)             # Ambil kelas prediksi (index probabilitas tertinggi)

y_true = final_test_gen.classes                              # Label asli (ground truth)

class_names = list(final_test_gen.class_indices.keys())      # Nama kelas (misalnya: glioma, meningioma, dll)

# ============================================================
# 3. HITUNG METRIK (TERMASUK SPECIFICITY)
# ============================================================

cm = confusion_matrix(y_true, y_pred_classes)                # Hitung confusion matrix

# Fungsi untuk menghitung specificity multi-class
def calculate_specificity(cm):
    specificities = []
    for i in range(len(cm)):
        tn = np.sum(cm) - (np.sum(cm[i, :]) + np.sum(cm[:, i]) - cm[i, i]) # True Negative
        fp = np.sum(cm[:, i]) - cm[i, i]                                   # False Positive
        spec = tn / (tn + fp) if (tn + fp) > 0 else 0                      # Rumus specificity
        specificities.append(spec)
    return np.mean(specificities)                                          # Rata-rata specificity semua kelas

# Hitung metrik utama
accuracy = accuracy_score(y_true, y_pred_classes)                          # Akurasi
precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred_classes, average='macro')
specificity = calculate_specificity(cm)                                    # Specificity

# Buat tabel metrik
metrics_table = pd.DataFrame({
    'Metric': ['Accuracy', 'Precision', 'Recall (Sensitivity)', 'Specificity', 'F1-Score'],
    'Score': [accuracy, precision, recall, specificity, f1]
})
metrics_table['Score (%)'] = metrics_table['Score'].apply(lambda x: f"{x*100:.2f}%")

print("\n=== TABEL PERFORMA LENGKAP ===")
from IPython.display import display, HTML
display(HTML(metrics_table.to_html(index=False, classes='table table-striped'))) # Tampilkan tabel rapi

# ============================================================
# 4. VISUALISASI CONFUSION MATRIX
# ============================================================

plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',          # Heatmap confusion matrix
            xticklabels=class_names,                        # Label kolom = nama kelas prediksi
            yticklabels=class_names)                        # Label baris = nama kelas asli
plt.xlabel('Prediksi Model (Predicted)')
plt.ylabel('Label Asli (Actual)')
plt.title('Confusion Matrix (4 Kelas)')
plt.show()

# ============================================================
# 5. VISUALISASI ROC CURVE (MULTI-CLASS)
# ============================================================

lb = LabelBinarizer()                                       # Binarize label untuk ROC (One-vs-Rest)
y_true_bin = lb.fit_transform(y_true)
n_classes = len(class_names)

fpr, tpr, roc_auc = dict(), dict(), dict()                  # Dictionary untuk FPR, TPR, AUC

for i in range(n_classes):                                  # Hitung ROC tiap kelas
    fpr[i], tpr[i], _ = roc_curve(y_true_bin[:, i], y_pred_probs[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

plt.figure(figsize=(10, 8))
colors = cycle(['blue', 'red', 'green', 'orange', 'purple']) # Warna berbeda tiap kelas

for i, color in zip(range(n_classes), colors):
    plt.plot(fpr[i], tpr[i], color=color, lw=2,
             label='ROC {0} (AUC = {1:0.4f})'.format(class_names[i], roc_auc[i]))

plt.plot([0, 1], [0, 1], 'k--', lw=2)                       # Garis diagonal baseline
plt.xlim([0.0, 1.0]); plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) - Multi-Class')
plt.legend(loc="lower right")
plt.grid(True)
plt.show()

In [ ]:
# ============================================================
# ANALISIS PERFORMA PER KELAS (DETIL)
# ============================================================

print("--- LAPORAN PERFORMA PER KELAS ---")

# 1. Generate Report Dictionary
# Output berupa dictionary agar bisa kita olah jadi Tabel
report_dict = classification_report(y_true, y_pred, target_names=class_names, output_dict=True)

# 2. Buat DataFrame dari Report
df_per_class = pd.DataFrame(report_dict).transpose()

# 3. Hitung Specificity Per Kelas
# Specificity = TN / (TN + FP)
# Rumus: Kemampuan menghindari False Alarm pada kelas tersebut
class_specificities = []
for i in range(len(cm)):
    tn = np.sum(np.delete(np.delete(cm, i, 0), i, 1)) 
    fp = np.sum(np.delete(cm[:, i], i))
    spec = tn / (tn + fp) if (tn + fp) > 0 else 0
    class_specificities.append(spec)

# 4. Rapikan Tabel
# Ambil hanya baris kelas (buang baris 'accuracy', 'macro avg', 'weighted avg' sementara)
df_classes = df_per_class.loc[class_names].copy()

# Masukkan kolom Specificity
df_classes['specificity'] = class_specificities

# Urutkan kolom agar enak dibaca
df_classes = df_classes[['precision', 'recall', 'specificity', 'f1-score', 'support']]

# Rename kolom biar lebih cantik
df_classes.columns = ['Precision', 'Sensitivity (Recall)', 'Specificity', 'F1-Score', 'Jumlah Data']

# Format angka menjadi persentase
for col in ['Precision', 'Sensitivity (Recall)', 'Specificity', 'F1-Score']:
    df_classes[col] = df_classes[col].apply(lambda x: f"{x*100:.2f}%")

# Tampilkan Tabel
from IPython.display import display, HTML
display(HTML(df_classes.to_html(classes='table table-striped table-hover')))

print("\nKeterangan:")
print("- Sensitivity (Recall): Seberapa hebat model menemukan tumor jenis ini.")
print("- Specificity: Seberapa hebat model tidak salah tuduh (misal: Pituitary dibilang Glioma).")
print("- Support: Jumlah gambar asli untuk kelas tersebut di data test.")

In [ ]:
# ============================================================
# FUNGSI GRAD-CAM 
# ============================================================
def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
    # 1. Buat Input Baru (Bersih)
    grad_model_input = tf.keras.Input(shape=(128, 128, 3))   # Input layer baru agar tidak warning

    # 2. Alirkan Input Baru ke Layer Lama (Re-Connect)
    x = grad_model_input
    last_conv_layer_output = None
    
    for layer in model.layers:                               # Loop semua layer model
        if isinstance(layer, tf.keras.layers.InputLayer):    # Skip InputLayer lama
            continue
        x = layer(x)                                         # Sambungkan layer ke input baru
        if layer.name == last_conv_layer_name:               # Jika layer = conv terakhir
            last_conv_layer_output = x                       # Simpan output conv terakhir
    
    grad_model_output = x                                    # Output akhir model

    # 3. Buat Model Grad-CAM Baru
    grad_model = tf.keras.models.Model(
        inputs=grad_model_input,
        outputs=[last_conv_layer_output, grad_model_output]  # Model keluarkan conv terakhir + prediksi
    )

    # 4. Hitung Gradien
    with tf.GradientTape() as tape:
        last_conv_output, preds = grad_model(img_array)      # Forward pass
        if pred_index is None:                               # Jika index kelas belum ditentukan
            pred_index = tf.argmax(preds[0])                 # Ambil kelas prediksi tertinggi
        class_channel = preds[:, pred_index]                 # Ambil channel prediksi kelas target

    grads = tape.gradient(class_channel, last_conv_output)   # Hitung gradien terhadap conv terakhir
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))     # Rata-rata gradien per filter
    
    last_conv_output = last_conv_output[0]                   # Ambil output conv untuk gambar pertama
    heatmap = last_conv_output @ pooled_grads[..., tf.newaxis] # Kombinasi gradien + feature map
    heatmap = tf.squeeze(heatmap)

    # 5. Normalisasi Heatmap
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap) # Normalisasi ke [0,1]
    return heatmap.numpy()

# ============================================================
# 3. FUNGSI VISUALISASI (DENGAN DETEKSI LAYER OTOMATIS)
# ============================================================
def display_gradcam_with_barplot(img_path):
    # --- A. Preprocessing ---
    img = cv2.imread(img_path)                               # Baca gambar
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)           # Konversi ke RGB
    img_resized = cv2.resize(img_rgb, (128, 128))            # Resize ke 128x128
    img_tensor = np.expand_dims(img_resized.astype('float32') / 255.0, axis=0) # Normalisasi + batch

    # --- B. Cari Layer Konvolusi Terakhir ---
    last_conv_layer_name = None
    for layer in reversed(model_final.layers):               # Loop layer dari belakang
        if 'conv' in layer.name.lower():                     # Cari layer conv terakhir
            last_conv_layer_name = layer.name
            break

    # --- C. Prediksi & Heatmap ---
    heatmap = make_gradcam_heatmap(img_tensor, model_final, last_conv_layer_name) # Buat heatmap
    preds = model_final.predict(img_tensor, verbose=0)       # Prediksi probabilitas
    pred_idx = np.argmax(preds[0])                           # Index kelas prediksi
    confidence = preds[0][pred_idx]                          # Confidence score
    class_names = list(final_test_gen.class_indices.keys())  # Nama kelas dari generator
    pred_label = class_names[pred_idx]                       # Label prediksi

    # --- D. Tampilkan (Overlay & Plot) ---
    heatmap_resized = cv2.resize(heatmap, (img_rgb.shape[1], img_rgb.shape[0])) # Resize heatmap
    heatmap_uint8 = np.uint8(255 * heatmap_resized)          # Konversi ke uint8
    heatmap_colored = cv2.applyColorMap(heatmap_uint8, cv2.COLORMAP_JET) # Warna heatmap
    overlay = cv2.addWeighted(img_rgb, 0.6, heatmap_colored, 0.4, 0)     # Overlay ke gambar asli

    plt.figure(figsize=(14, 6))

    # Gambar 1: Grad-CAM Overlay
    plt.subplot(1, 2, 1)
    plt.imshow(overlay)
    plt.title(f"Prediksi: {pred_label.upper()} ({confidence*100:.2f}%)")
    plt.axis('off')
    plt.xlabel("Area Merah = Fitur Kunci")

    # Gambar 2: Bar Plot Probabilitas
    plt.subplot(1, 2, 2)
    colors = ['lightgray'] * len(class_names)
    is_tumor = 'normal' not in pred_label.lower() and 'no' not in pred_label.lower()
    colors[pred_idx] = 'red' if is_tumor else 'green'        # Warna merah jika tumor, hijau jika normal
    
    y = np.arange(len(class_names))
    plt.barh(y, preds[0], color=colors)                      # Bar plot probabilitas
    plt.yticks(y, class_names)
    plt.xlim(0, 1.0)
    plt.title("Probabilitas Model")
    
    for i, v in enumerate(preds[0]):                         # Tambahkan nilai % di bar plot
        plt.text(v + 0.01, i, f"{v*100:.1f}%", va='center', fontweight='bold')

    plt.tight_layout()
    plt.show()

# ============================================================
# 4. EKSEKUSI
# ============================================================
if len(df_test_final) > 0:
    random_row = df_test_final.sample(1).iloc[0]             # Ambil 1 sample random dari test set
    target_path = random_row['filename']                     # Path gambar
    label_asli = random_row['class']                         # Label asli
    
    print(f"--- ANALISIS GRAD-CAM ---")
    print(f"File: {target_path}")
    print(f"Label Asli: {label_asli}")
    
    display_gradcam_with_barplot(target_path)                # Tampilkan hasil Grad-CAM
else:
    display_gradcam_with_barplot(IMAGE_PATH)                 # Fallback ke path manual jika test kosong